In [1]:
from cw_api import all_tickets

# PARSE DATA INTO DICTS WITH SUMMARY AND STATUS

# extract pto summaries from api response
pto_data = []
for ticket in all_tickets:
    if "summary" in ticket and "status" in ticket and isinstance(ticket["status"], dict) and ticket["status"]["name"] != ">Closed":
        if "name" in ticket["status"]:
            pto_data.append({
                "summary": ticket["summary"],
                "status": ticket["status"]["name"]
            })


print(pto_data)

[{'summary': 'David L (Remote/FL Escalations) Personal 6/11', 'status': 'Needs Approval'}, {'summary': 'Khadijia B (NYC/HHAR - HelpDesk) FMLA 6/12', 'status': 'Needs Approval'}, {'summary': 'Eldan B (Philippines - Help Desk) Sick 6/09', 'status': 'Added to LCS Calendar'}, {'summary': 'Arinze M (NYC - Help Desk) FMLA 8/25, 8/26, 8/27, 8/28, 8/29', 'status': 'Added to LCS Calendar'}, {'summary': 'Arinze M (NYC - Help Desk) FMLA 8/08, 8/11, 8/12, 8/13, 8/14, 8/15, 8/18, 8/19, 8/20, 8/21, 8/22', 'status': 'Added to LCS Calendar'}, {'summary': 'Arinze M (NYC - Help Desk) FMLA 7/24, 7/25, 7/28, 7/29, 7/30, 7/31, 8/01, 8/04, 8/05, 8/06, 8/07', 'status': 'Added to LCS Calendar'}, {'summary': 'Arinze M (NYC - Help Desk) FMLA 7/09, 7/10, 7/11, 7/14, 7/15, 7/16, 7/17, 7/18, 7/21, 7/22, 7/23', 'status': 'Added to LCS Calendar'}, {'summary': 'Arinze M (NYC - Help Desk) FMLA 6/24, 6/25, 6/26, 6/27, 6/30, 7/01, 7/02, 7/03, 7/04, 7/07, 7/08', 'status': 'Added to LCS Calendar'}, {'summary': 'Dan L (Hic

In [ ]:
# PARSE OUT NAME, LOCATION, TEAM, DATE(S), TIME FROM PTO_DATA

import re

pattern = re.compile(r"""
    ^                             # start of string
    (?P<Name>[A-Za-z]+\s[A-Za-z]) # Name: First Last (2 words)
    \s\(
        (?P<Location>[^-]+?)      # Location: everything before " -"
        \s*-\s*
        (?P<Team>[^)]+)           # Team: everything before ")"
    \)\s*
    (?P<Type>[A-Za-z]+)?          # Optional Type (e.g., "FMLA", "PTO", etc.)
    (?:\s*\((?P<TimeRange>\d{1,2}:\d{2}[ap] - \d{1,2}:\d{2}[ap])\))? # Optional time range
    \s*
    (?P<Days>(?:\d{1,2}/\d{1,2})(?:,\s*\d{1,2}/\d{1,2})*) # One or more dates
    $                             # end of string
""", re.VERBOSE | re.IGNORECASE)


parsed_pto_data = []

for entry in pto_data:
    summary = entry.get('summary', '')
    match = pattern.match(summary)

    if match:
        parsed = match.groupdict()
        parsed['status'] = entry.get('status')
        parsed_pto_data.append(parsed)

print(parsed_pto_data)

[{'Name': 'Khadijia B', 'Location': 'NYC/HHAR', 'Team': 'HelpDesk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '6/12', 'status': 'Needs Approval'}, {'Name': 'Eldan B', 'Location': 'Philippines', 'Team': 'Help Desk', 'Type': 'Sick', 'TimeRange': None, 'Days': '6/09', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location': 'NYC', 'Team': 'Help Desk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '8/25, 8/26, 8/27, 8/28, 8/29', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location': 'NYC', 'Team': 'Help Desk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '8/08, 8/11, 8/12, 8/13, 8/14, 8/15, 8/18, 8/19, 8/20, 8/21, 8/22', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location': 'NYC', 'Team': 'Help Desk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '7/24, 7/25, 7/28, 7/29, 7/30, 7/31, 8/01, 8/04, 8/05, 8/06, 8/07', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location': 'NYC', 'Team': 'Help Desk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '7/09

In [10]:
# SPLIT DATA INTO PTO HAPPENING AND PTO REQUESTED

pto_happening = []
pto_requests = []

for entry in parsed_pto_data:
    if entry['status'] in ["Approved", "Added to LCS Calendar", "Time Checked"]:
        pto_happening.append(entry)
    elif entry['status'] == 'Needs Approval':
        pto_requests.append(entry)

print(f'PTO happening is {pto_happening}')
print(f'PTO requested is {pto_requests}')

PTO happening is [{'Name': 'Eldan B', 'Location': 'Philippines', 'Team': 'Help Desk', 'Type': 'Sick', 'TimeRange': None, 'Days': '6/09', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location': 'NYC', 'Team': 'Help Desk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '8/25, 8/26, 8/27, 8/28, 8/29', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location': 'NYC', 'Team': 'Help Desk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '8/08, 8/11, 8/12, 8/13, 8/14, 8/15, 8/18, 8/19, 8/20, 8/21, 8/22', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location': 'NYC', 'Team': 'Help Desk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '7/24, 7/25, 7/28, 7/29, 7/30, 7/31, 8/01, 8/04, 8/05, 8/06, 8/07', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location': 'NYC', 'Team': 'Help Desk', 'Type': 'FMLA', 'TimeRange': None, 'Days': '7/09, 7/10, 7/11, 7/14, 7/15, 7/16, 7/17, 7/18, 7/21, 7/22, 7/23', 'status': 'Added to LCS Calendar'}, {'Name': 'Arinze M', 'Location'

two ways to display data
one is by date
so for 7/11 we see everyone already off and everyone requesting off

then by request
click on a time off request to see who else is off on those days
this one seems more useful